# dependencies
connect to cpu if want to use behaviour cloning

environment interaction numbers modified and aligned

rerun complete - 2024-03-12

In [1]:
#!git clone http://github.com/HumanCompatibleAI/imitation
#!pip install -e "/content/imitation[dev]"

In [2]:
!pip install gymnasium[mujoco]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.8/211.8 kB 13.3 MB/s eta 0:00:00


In [3]:
!pip install imitation
!pip install seals
!pip install shimmy
!pip install git+https://github.com/Stable-Baselines-Team/stable-baselines3-contrib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.4/216.4 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.7/181.7 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.9/107.9 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 379.9/379.9 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 11.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 42.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/8

# environment and expert

In [4]:
import gymnasium as gym
import imitation
import seals

In [5]:
import numpy as np
from imitation.policies.serialize import load_policy
from imitation.util.util import make_vec_env
from imitation.data.wrappers import RolloutInfoWrapper
from imitation.algorithms.adversarial.gail import GAIL
from imitation.rewards.reward_nets import BasicRewardNet
from imitation.util.networks import RunningNorm
from stable_baselines3 import PPO
from stable_baselines3.ppo import MlpPolicy, CnnPolicy
from stable_baselines3.common.evaluation import evaluate_policy
import sb3_contrib
from sb3_contrib import TRPO
from imitation.data import rollout
SEED = 42
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [6]:
def make_env_expert_rollouts(n_trajs):
    # Create the environment, load expert policy, and sample trajectories
    # from the expert policy and environment.
    global SEED
    env = make_vec_env(
        "seals/Walker2d-v1", # seals:seals/CartPole-v0
        rng=np.random.default_rng(SEED),
        n_envs=n_trajs,
        post_wrappers=[
            lambda env, _: RolloutInfoWrapper(env)
        ],  # needed for computing rollouts later
        parallel=True,
    )
    expert = load_policy(
        "ppo-huggingface",
        organization="HumanCompatibleAI",
        env_name="seals/Walker2d-v1",
        venv=env,
    )
    rollouts = rollout.rollout(
        expert,
        env,
        rollout.make_sample_until(min_timesteps=1000, min_episodes = n_trajs),
        rng=np.random.default_rng(SEED), #The random state to use for sampling trajectories,
    )

    return env, expert, rollouts


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [7]:
from typing import Callable, Dict, List, Optional, Tuple, Type, Union

from gymnasium import spaces
import torch as th
from torch import nn

from stable_baselines3 import PPO
from stable_baselines3.common.policies import ActorCriticPolicy


class Mlp3ReLUNetwork(nn.Module):
    """
    Custom network for policy and value function.
    It receives as input the features extracted by the features extractor.

    :param feature_dim: dimension of the features extracted with the features_extractor (e.g. features from a CNN)
    :param last_layer_dim_pi: (int) number of units for the last layer of the policy network
    :param last_layer_dim_vf: (int) number of units for the last layer of the value network
    """

    def __init__(
        self,
        feature_dim: int,
        last_layer_dim_pi: int = 64,
        last_layer_dim_vf: int = 64,
    ):
        super().__init__()

        # IMPORTANT:
        # Save output dimensions, used to create the distributions
        self.latent_dim_pi = last_layer_dim_pi
        self.latent_dim_vf = last_layer_dim_vf

        # Policy network
        self.policy_net = nn.Sequential(
            nn.Linear(feature_dim, last_layer_dim_pi), nn.Tanh(),
            nn.Linear(last_layer_dim_pi, last_layer_dim_pi), nn.Tanh(),
            nn.Linear(last_layer_dim_pi, last_layer_dim_pi), nn.Tanh(),
        )
        # Value network
        self.value_net = nn.Sequential(
            nn.Linear(feature_dim, last_layer_dim_vf), nn.Tanh(),
            nn.Linear(last_layer_dim_pi, last_layer_dim_pi), nn.Tanh(),
            nn.Linear(last_layer_dim_pi, last_layer_dim_pi), nn.Tanh(),
        )

    def forward(self, features: th.Tensor) -> Tuple[th.Tensor, th.Tensor]:
        """
        :return: (th.Tensor, th.Tensor) latent_policy, latent_value of the specified network.
            If all layers are shared, then ``latent_policy == latent_value``
        """
        return self.forward_actor(features), self.forward_critic(features)

    def forward_actor(self, features: th.Tensor) -> th.Tensor:
        return self.policy_net(features)

    def forward_critic(self, features: th.Tensor) -> th.Tensor:
        return self.value_net(features)


class Mlp3ReLUPolicy(ActorCriticPolicy):
    def __init__(
        self,
        observation_space: spaces.Space,
        action_space: spaces.Space,
        lr_schedule: Callable[[float], float],
        *args,
        **kwargs,
    ):
        # Disable orthogonal initialization
        kwargs["ortho_init"] = False
        super().__init__(
            observation_space,
            action_space,
            lr_schedule,
            # Pass remaining arguments to base class
            *args,
            **kwargs,
        )


    def _build_mlp_extractor(self) -> None:
        self.mlp_extractor = Mlp3ReLUNetwork(self.features_dim)

In [8]:
def make_PPO_learner_and_trainer(env, expert, rollouts):
    global SEED
# 可以调policy network的架构
# Custom actor (pi) and value function (vf) networks
# of two layers of size 32 each with Relu activation function
#policy_kwargs = dict(activation_fn=th.nn.ReLU,
#                     net_arch=[dict(pi=[32, 32], vf=[32, 32])])
    learner_PPO = PPO(
        env=env,
        policy=Mlp3ReLUPolicy,
        batch_size=128,
        clip_range=0.4,
        ent_coef= 0.0007566389899529574,
        learning_rate=1e-4, #1.943992487657563e-5
        gae_lambda=0.92,
        max_grad_norm=0.6,
        gamma=0.98,
        n_epochs=20,
        seed=SEED,
    )
    reward_net = BasicRewardNet(
        observation_space=env.observation_space,
        action_space=env.action_space,
        normalize_input_layer=RunningNorm,
    )
    gail_trainer = GAIL(
        demonstrations = rollouts, # expert trajectories
        demo_batch_size = 512,
        gen_replay_buffer_capacity = 16384,
        n_disc_updates_per_round = 16, #
        venv = env,
        gen_algo = learner_PPO,
        reward_net=reward_net,
        allow_variable_horizon=True,
        # gen_train_timesteps = 1000
        init_tensorboard=True,
        init_tensorboard_graph=True,

    )
    return learner_PPO, gail_trainer

# GAIL-PPO 1 trajectory

In [ ]:
env_1, expert_1, rollouts_1 = make_env_expert_rollouts(1)

In [ ]:
learner_PPO_1, gail_trainer_PPO_1 = make_PPO_learner_and_trainer(env_1, expert_1, rollouts_1)

Running with `allow_variable_horizon` set to True. Some algorithms are biased towards shorter or longer episodes, which may significantly confound results. Additionally, even unbiased algorithms can exploit the information leak from the termination condition, producing spuriously high performance. See https://imitation.readthedocs.io/en/latest/getting-started/variable-horizon.html for more information.


In [ ]:
expert_reward, _ = evaluate_policy(
    expert_1, env_1, 20, return_episode_rewards=True
)
print(
    "expert rewards after training:",
    np.mean(expert_reward),
    "+/-",
    np.std(expert_reward),
)

expert rewards after training: 2481.58420425 +/- 210.94854271462816


In [ ]:
env_1.seed(SEED)
learner_rewards_before_training, _ = evaluate_policy(
    learner_PPO_1, env_1, 20, return_episode_rewards=True
)

In [ ]:
gail_trainer_PPO_1.train(1000000)
# An upper bound on the number of transitions to sample from the environment during training.

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
env_1.seed(SEED)
learner_rewards_after_training, _ = evaluate_policy(
    learner_PPO_1, env_1, 20, return_episode_rewards=True
)

In [ ]:
print(
    "Rewards before training:",
    np.mean(learner_rewards_before_training),
    "+/-",
    np.std(learner_rewards_before_training),
)
print(
    "Rewards after training:",
    np.mean(learner_rewards_after_training),
    "+/-",
    np.std(learner_rewards_after_training),
)

Rewards before training: 82.1197979 +/- 1.917387081086781
Rewards after training: 619.38190165 +/- 246.51631973600556


# GAIL-PPO 4 trajectory

In [9]:
env_4, expert_4, rollouts_4 = make_env_expert_rollouts(4)
learner_PPO_4, gail_trainer_PPO_4 = make_PPO_learner_and_trainer(env_4, expert_4, rollouts_4)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


ppo-seals-Walker2d-v1.zip:   0%|          | 0.00/1.75M [00:00<?, ?B/s]

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

Running with `allow_variable_horizon` set to True. Some algorithms are biased towards shorter or longer episodes, which may significantly confound results. Additionally, even unbiased algorithms can exploit the information leak from the termination condition, producing spuriously high performance. See https://imitation.readthedocs.io/en/latest/getting-started/variable-horizon.html for more information.


In [ ]:
expert_reward, _ = evaluate_policy(
    expert_4, env_4, 20, return_episode_rewards=True
)
print(
    "expert rewards after training:",
    np.mean(expert_reward),
    "+/-",
    np.std(expert_reward),
)

expert rewards after training: 2583.1990069999997 +/- 173.87006932840748


In [10]:
env_4.seed(SEED)
learner_rewards_before_training, _ = evaluate_policy(
    learner_PPO_4, env_4, 20, return_episode_rewards=True
)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [11]:
print(
    "Rewards before training:",
    np.mean(learner_rewards_before_training),
    "+/-",
    np.std(learner_rewards_before_training),
)

Rewards before training: 115.23938380000001 +/- 70.06170595618798


In [ ]:
gail_trainer_PPO_4.train(3600000) #进度条显示的是gen_train_timestep除以这里的input

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
env_4.seed(SEED)
learner_rewards_after_training, _ = evaluate_policy(
    learner_PPO_4, env_4, 20, return_episode_rewards=True
)

In [ ]:
print(
    "learner rewards after training:",
    np.mean(learner_rewards_after_training),
    "+/-",
    np.std(learner_rewards_after_training),
)

learner rewards after training: 2237.9023784499996 +/- 282.9963647230639


# GAIL-PPO 10 trajectory

In [12]:
env_10, expert_10, rollouts_10 = make_env_expert_rollouts(10)
learner_PPO_10, gail_trainer_PPO_10 = make_PPO_learner_and_trainer(env_10, expert_10, rollouts_10)

Running with `allow_variable_horizon` set to True. Some algorithms are biased towards shorter or longer episodes, which may significantly confound results. Additionally, even unbiased algorithms can exploit the information leak from the termination condition, producing spuriously high performance. See https://imitation.readthedocs.io/en/latest/getting-started/variable-horizon.html for more information.


In [ ]:
print(rollouts_10.count)

<built-in method count of list object at 0x78d520878d00>


In [ ]:
expert_reward, _ = evaluate_policy(
    expert_10, env_10, 20, return_episode_rewards=True
)
print(
    "expert rewards after training:",
    np.mean(expert_reward),
    "+/-",
    np.std(expert_reward),
)

expert rewards after training: 2551.0492796 +/- 203.9012591006724


In [13]:
env_10.seed(SEED)
learner_rewards_before_training, _ = evaluate_policy(
    learner_PPO_10, env_10, 20, return_episode_rewards=True
)

In [14]:
print(
    "Rewards before training:",
    np.mean(learner_rewards_before_training),
    "+/-",
    np.std(learner_rewards_before_training),
)

Rewards before training: 104.6138885 +/- 80.13520475144537


In [ ]:
gail_trainer_PPO_10.train(3600000) #进度条显示的是gen_train_timestep除以这里的input

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
env_10.seed(SEED)
learner_rewards_after_training, _ = evaluate_policy(
    learner_PPO_10, env_10, 20, return_episode_rewards=True
)

In [ ]:
print(
    "learner rewards after training:",
    np.mean(learner_rewards_after_training),
    "+/-",
    np.std(learner_rewards_after_training),
)

learner rewards after training: 2457.6739203500006 +/- 183.8450179004427
